# Exploring weather data
This notebook loads and checks the format of various weather data, before compiling a list of any missing dates for download. First we load required dependencies and specify the location of files.

You might have to install the eccodes package to be able to load both .grib and .nc file types through xarray - the env_eccodes.txt file can be used to set up a (conda) environment for this.

In [5]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import cfgrib
import zipfile
tux_loc = r'N:\National Gas Transmission\15928 - Short Term Probabilistic forecasting for National Gas\2 - Client Data'


In [6]:
def remove_empty_values(d):
    keys_to_remove = [key for key, value in d.items() if not value]
    for key in keys_to_remove:
        del d[key]

In [8]:
# This function returns list of files both grib and nc.

#path to the folder where you will find these year folders containing grib or nc files
path_to_folder = r'N:\National Gas Transmission\15928 - Short Term Probabilistic forecasting for National Gas\2 - Client Data'

# add other extensions with (.) in  the tuple if files other than grib or nc types are also needed
extensions_to_choose = ('.nc','grib')

#year of folders to choose
range_of_years = np.arange(2019,2020)
#files_dict ={}
def files_list(path_to_folder, extensions_to_choose, range_of_years):
    short_horizons = range_of_years
    extensions_file = extensions_to_choose
    files_dict = {}
    for year in short_horizons:
        files = os.listdir(path_to_folder+'/ECMWF/GB_hres_Domain/'+str(year))
        if year in range_of_years:
             grib_files = [f for f in files if f.endswith(extensions_file)]
             files_dict[year] = grib_files
             
    remove_empty_values(files_dict)
    return files_dict
files_dict = files_list(path_to_folder,extensions_to_choose,range_of_years)
subset_files_2019 = files_dict.get(2019)[:2]

files_dict = {2019: subset_files_2019}
files_dict

{2019: ['hres-UKNS_20190418.grib', 'hres-UKNS_20191231.grib']}

In [9]:
year = 2019
ds = xr.open_dataset('N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2019/hres-UKNS_20190418.grib')
ds1 = xr.open_dataset('N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2019/hres-UKNS_20190418.grib')


Ignoring index file 'N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2019/hres-UKNS_20190418.grib.923a8.idx' incompatible with GRIB file
Ignoring index file 'N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2019/hres-UKNS_20190418.grib.923a8.idx' incompatible with GRIB file


In [4]:
import xarray as xr

# Initialize an empty list to store datasets for each year
year_datasets = []

# Loop through each year and file paths in the files_dict
for year, file_names in files_dict.items():
    # Initialize an empty list to store datasets for the current year
    year_datasets_list = []
    
    # Loop through each file path for the current year
    for file_name in file_names:
        # Read the dataset for the current file using xr.open_dataset()
        ds = xr.open_dataset.open_dataset(tux_loc + '/ECMWF/GB_hres_Domain/' + str(year) + '/' + file_name)
        #ds = ds[["t2m","u10","v10"]].sel(time=slice(ds.coords['time'][0],ds.coords['time'][0] + np.timedelta64(1, 'h')))
        #extract locations for latitude ,longitude,
        # add a column for issue time for each ds
        # convert that target time by issue time + step
        #check 2017,2018 mixed format ...
        # sqrt root of summation of squares (10 and 100 for v and u)
        # pythagors theoram for wind direction... angles..
        # zero degrees corresponding to north.. clock wise reading.. 
        year_datasets_list.append(ds)
    
    # Merge all the datasets for the current year into one final dataset
    #concat with pandas, remove this merge of Xarray
    merged_year_ds = xr.merge(year_datasets_list)
    
    
    # Append the merged dataset for the current year to the main list
    year_datasets.append(merged_year_ds)

# Merge all the datasets for different years into one final dataset
merged_ds = xr.merge(year_datasets)
print(merged_ds)


NameError: name 'files_dict' is not defined

In [295]:
files_dict

{2017: ['hres-UKNS_20170409.nc', 'hres-UKNS_20170203.nc']}

In [232]:
files = ['hres-UKNS_20170101.nc']
list_df_issues = []
for file in files:
    print(file)
    #issue_time = pd.to_datetime("hres-UKNS_20170101.nc",utc=True).to_datetime64()
        # print(issue_time)
    date_string = file.replace('hres-UKNS_', '').replace('.nc', '')
        # Parse the date string to a datetime object
    issue_time = datetime.strptime(date_string, "%Y%m%d")
        
       #ds = xr.open_dataset(f'{year}/{file}')
    ds =  cfgrib.open_dataset("hres-UKNS_20170101.nc")
      #("N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2019/hres-UKNS_20191229.grib") 
        # No need to interpolate to greater time resoultion here... (save memoy and do it only when required)
        # times_out = np.arange(ds.coords['time'].values[0],
        #                       ds.coords['time'].values[0] + np.timedelta64(144, 'h'),
        #                       dtype='datetime64[h]')

        # Only take lead-times up to 144
    #print(ds)
    #data_df = ds.to_dataframe()
    #data_df.reset_index(inplace=True)
    #csv_file_path = 't1.csv'
    #data_df.to_csv(csv_file_path, index=False)
    #ds = ds[["t2m","u10","v10"]].sel(time=slice(ds.coords['time'][0],ds.coords['time'][0] + np.timedelta64(144, 'h')))
    print(ds)
    #data_df = ds.to_dataframe()
    #data_df.reset_index(inplace=True)
    #csv_file_path = 't2.csv'
    #data_df.to_csv(csv_file_path, index=False)
        # Interpolate to weather stations
    list_df_locations = []
    for i in range(len(locations_temp)):
        ds_temp = ds.interp(longitude=locations_temp.iloc[i]['longitude'],latitude=locations_temp.iloc[i]['latitude'])
        df_temp = ds_temp.to_dataframe()
        #df_temp = df_temp.drop(['longitude','latitude'],axis=1)
        df_temp['location'] = locations_temp.iloc[i]['name']
        list_df_locations.append(df_temp)
        
        df_single_issue = pd.concat(list_df_locations)
        df_single_issue['issue_time'] = issue_time
        
        list_df_issues.append(df_single_issue)
   #dfs = ds.to_dataframe()
    df_year = pd.concat(list_df_issues)
    df_year.reset_index().to_feather(f'tigge_v011_{year}.feather')

hres-UKNS_20170101.nc
<xarray.Dataset>
Dimensions:     (time: 2, step: 91, latitude: 91, longitude: 101)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2017-01-01 2017-01-01T12:00:00
  * step        (step) timedelta64[ns] 00:00:00 01:00:00 ... 3 days 18:00:00
    surface     float64 ...
  * latitude    (latitude) float64 59.0 58.9 58.8 58.7 ... 50.3 50.2 50.1 50.0
  * longitude   (longitude) float64 -7.0 -6.9 -6.8 -6.7 -6.6 ... 2.7 2.8 2.9 3.0
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    u10         (time, step, latitude, longitude) float32 ...
    v10         (time, step, latitude, longitude) float32 ...
    t2m         (time, step, latitude, longitude) float32 ...
    d2m         (time, step, latitude, longitude) float32 ...
    ssrd        (time, step, latitude, longitude) float32 ...
    lcc         (time, step, latitude, longitude) float32 ...
    mcc         (time, step, latitude, longitude) float32 ...
    hcc         (time, s

In [233]:
df = pd.read_feather('tigge_v011_2021.feather')
df
df.size
csv_file_path = 't12.csv'
df.to_csv(csv_file_path, index=False)

In [235]:
df.head()


,time,step,number,surface,valid_time,u10,v10,t2m,d2m,ssrd,lcc,mcc,hcc,tp,u100,v100,longitude,latitude,location,issue_time
0,2017-01-01,0 days 00:00:00,0,0.0,2017-01-01 00:00:00,2.210841,3.693239,279.433120,278.607553,0.0,0.852562,0.009667,0.000000,0.000000,4.281644,6.346291,-0.449,51.479,Heathrow,2017-01-01
1,2017-01-01,0 days 01:00:00,0,0.0,2017-01-01 01:00:00,1.975294,3.678520,278.930735,277.912622,0.0,0.980756,0.112118,0.030484,0.000006,3.873738,6.644033,-0.449,51.479,Heathrow,2017-01-01
2,2017-01-01,0 days 02:00:00,0,0.0,2017-01-01 02:00:00,2.053827,3.639928,278.576111,277.667886,0.0,1.000000,0.591768,0.725935,0.000034,4.240539,6.456833,-0.449,51.479,Heathrow,2017-01-01
3,2017-01-01,0 days 03:00:00,0,0.0,2017-01-01 03:00:00,1.789081,3.891595,278.831056,277.676591,0.0,0.971054,0.574940,0.728468,0.000074,3.766155,6.728262,-0.449,51.479,Heathrow,2017-01-01
4,2017-01-01,0 days 04:00:00,0,0.0,2017-01-01 04:00:00,2.663051,3.980583,279.229790,277.860031,0.0,0.798193,0.189124,0.928884,0.000080,5.278740,6.693228,-0.449,51.479,Heathrow,2017-01-01


In [149]:
df.step.tail()

1906   3 days 14:00:00
1907   3 days 15:00:00
1908   3 days 16:00:00
1909   3 days 17:00:00
1910   3 days 18:00:00
Name: step, dtype: timedelta64[ns]

First we check the ECMWF HRES forecasts. Horizons out to 48hrs ahead are in folders labelled with the year. We check that a file exists for every date in the year and record any missing days. The 2017 and 2018 forecasts have been saved with a .nc file extension but are actually .grib files so these are loaded in the same way, but care must be taken to ensure the correct files are selected.

In [67]:
short_horizons = np.arange(2014,2022) # for now
missing_files = np.empty((0,), dtype='datetime64[us]')
short_horizons = np.arange(2017,2022) # DELETE once folders unzipped
for year in short_horizons:
    files = os.listdir(tux_loc+'/ECMWF/GB_hres_Domain/'+str(year))

    if year in [2017, 2018]:
        grib_files = [f for f in files if f.endswith('.nc')] # mis-named as .nc
        file_dates = pd.DataFrame({'datestring': [ x.replace('hres-UKNS_', '').replace('.nc', '') for x in grib_files]})
    else:
        grib_files = [f for f in files if f.endswith('.grib')]
        file_dates = pd.DataFrame({'datestring': [ x.replace('hres-UKNS_', '').replace('.grib', '') for x in grib_files]})

    file_dates['year'] = file_dates.datestring.str[:4].astype(int)
    file_dates['month'] = file_dates.datestring.str[4:6].astype(int)
    file_dates['day'] = file_dates.datestring.str[6:].astype(int)
    file_dates['datetime'] = pd.to_datetime(file_dates[['year', 'month', 'day']])
    complete_dates = pd.date_range(start=f'{year}/1/1', end=f'{year}/12/31')
    if len(complete_dates)!= len(file_dates.datetime):
        missing_dates = np.setdiff1d(complete_dates, file_dates.datetime)
        missing_files = np.append(missing_files, missing_dates)

In [68]:
if len(missing_files)>0:
    missing_files = pd.DataFrame({'datetime': missing_files})
    missing_files['year'] = missing_files['datetime'].dt.year
    missing_files['month'] = missing_files['datetime'].dt.month
    missing_files['day'] = missing_files['datetime'].dt.day
    missing_files['datestring'] = missing_files['year'].astype(str) + \
                                  missing_files['month'].astype(str).str.zfill(2) + \
                                  missing_files['day'].astype(str).str.zfill(2)
    missing_files.to_csv(f'{tux_loc}/ECMWF/gb_hres_Domain/missing_dates.csv', index=False)

And let's have a look at one file to see the structure:

In [ ]:
#year = "2017"
#data = cfgrib.open_dataset(f'{tux_loc}/ECMWF/GB_hres_Domain/{year}/{grib_files[0]}')
data = cfgrib.open_dataset("N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2019/hres-UKNS_20191229.grib")


data1 = cfgrib.open_dataset("N:/National Gas Transmission/15928 - Short Term Probabilistic forecasting for National Gas/2 - Client Data/ECMWF/GB_hres_Domain/2017/hres-UKNS_20170101.nc")

 
#files = os.listdir(tux_loc+'/ECMWF/GB_hres_Domain/2017')

#netcdf_files = [f for f in files if f.endswith('.nc')]
#print(f'{tux_loc}/ECMWF/GB_hres_Domain/2017/{netcdf_files[0]}')
#data1 = xr.open_mfdataset(f'{tux_loc}/ECMWF/GB_hres_Domain/2017/test.nc',engine='pseudonetcdf')
#datdataa1 = xr.open_dataset(f'{tux_loc}/ECMWF/GB_hres_Domain/2017/', engine = 'netcdf4')


target = data.time + data.step
data.coords['target'] = target

target = data1.time + data1.step
data1.coords['target'] = target


merged_ds = xr.merge([data, data1])

data_df = merged_ds.to_dataframe()
data_df.reset_index(inplace=True)
csv_file_path = 'target_merged.csv'
data_df.to_csv(csv_file_path, index=False)



# Print the new DataArray
#print(data_df.head())

2

In [99]:
#get time series for at all locations for one variable u10
#get time series for same locations at one point for series of variables
#get time series for one location at single point
# get time series for range of dates and range of longitude and latitude
#get time series for other coordinates like target time 


# Calculate the area by multiplying length and breadth
target = data.time + data.step
data.coords['target'] = target


# Print the new DataArray
#print(data.head())


#data_df = data.to_dataframe()
#data_df.reset_index(inplace=True)
#csv_file_path = 'target1.csv'
#data_df.to_csv(csv_file_path, index=False)






In [84]:
#data_df = data.to_dataframe()
#data_df.reset_index(inplace=True)
#csv_file_path = 'target1.csv'
#data_df.to_csv(csv_file_path, index=False)






We can see that our data has 4 main dimensions: latitude, longitude, (issue) time, and step (forecast horizon). As well as that, it contains 10 weather variables. These are detailed in the data shopping list document and at https://codes.ecmwf.int/grib/param-db/?filter=grib1.

If we want to select out a set of variables at a certain location and set of time horizons we can do, for example:

In [8]:
data1.to_csv("ravi.csv", index=False)

In [ ]:
data[['u100', 'v100']].sel(latitude=58.8, longitude=-6.9)

## ERA5 reanalysis data
Let's also check the times available for ERA5 reanalysis in the same way. These are saved as a file per month.

In [69]:

files = os.listdir(tux_loc+'/ECMWF/ERA5/GB_era5/p4r')
netcdf_files = [f for f in files if f.endswith('.nc')]

file_dates = pd.DataFrame({'datestring': [ x.replace('era5-GB_', '').replace('.nc', '') for x in netcdf_files]})
file_dates['year'] = file_dates.datestring.str[:4].astype(int)
file_dates['month'] = file_dates.datestring.str[4:].astype(int)
start_year = file_dates.iloc[0]['year']
start_month = file_dates.iloc[0]['month']
end_year = file_dates.iloc[-1]['year']
end_month = file_dates.iloc[-1]['month']
month_ts = pd.date_range(start=f'{start_year}-{start_month}', end=f'{end_year}-{end_month}', freq='MS')
# file_dates['day'] = file_dates.datestring.str[6:].astype(int)
# file_dates['datetime'] = pd.to_datetime(file_dates[['year', 'month', 'day']])
# complete_dates = pd.date_range(start=f'{year}/1/1', end=f'{year}/12/31')

In [72]:
# # load in and check file
#!pip install "dask[complete]"
ERA5 = xr.open_mfdataset(f'{tux_loc}/ECMWF/ERA5/GB_era5/p4r/{netcdf_files[0]}', engine='netcdf4')
test=2

# check for and aggregate over expver dimension
if 'expver' in ERA5.dims:
    ERA5 = ERA5.sel(expver=1).combine_first(ERA5.sel(expver=5))
ERA5

<xarray.Dataset>
Dimensions:    (longitude: 41, latitude: 37, time: 720)
Coordinates:
  * longitude  (longitude) float32 -7.0 -6.75 -6.5 -6.25 ... 2.25 2.5 2.75 3.0
  * latitude   (latitude) float32 59.0 58.75 58.5 58.25 ... 50.5 50.25 50.0
  * time       (time) datetime64[ns] 2020-04-01 ... 2020-04-30T23:00:00
Data variables:
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    i10fg      (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    sd         (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    sf         (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    swvl1      (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
    swvl2      (time, latitude, longitude) float32 dask.array<chunksize=(720, 37, 41), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-02-12 01:49:56 GMT by grib_to_netcdf-2.24.2: /opt/ecmw...